In [1]:
import numpy as np
import pandas as pd
import lightgbm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
import math
from sklearn import metrics
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import os
import glob

In [2]:

data_path='/home/madnisal/Documents/ML_Project/datasets/'
training_df = pd.read_csv(data_path+'train.csv', index_col="tripid")

In [3]:
def dist_from_coordinates(lat1, lon1, lat2, lon2):
  R = 6371  # Earth radius in km

  #conversion to radians
  d_lat = np.radians(lat2-lat1)
  d_lon = np.radians(lon2-lon1)

  r_lat1 = np.radians(lat1)
  r_lat2 = np.radians(lat2)

  #haversine formula
  a = np.sin(d_lat/2.) **2 + np.cos(r_lat1) * np.cos(r_lat2) * np.sin(d_lon/2.)**2

  haversine = 2 * R * np.arcsin(np.sqrt(a))

  return haversine

In [4]:
training_df['pickup_time'] = pd.to_datetime(training_df['pickup_time'], format="%m/%d/%Y %H:%M")
training_df['drop_time'] = pd.to_datetime(training_df['drop_time'], format="%m/%d/%Y %H:%M")

In [5]:
training_df = training_df.assign(timeOfDay=pd.cut(training_df.pickup_time.dt.hour,[-1, 8, 20, 24],labels=['dawn','day', 'night']))

In [6]:
training_df.loc[training_df['timeOfDay'] == 'day', 'isNormalCharge'] = 1
training_df.loc[training_df['timeOfDay'] != 'day', 'isNormalCharge'] = 0

In [7]:
durations = []
for index,row in training_df.iterrows():
  provided_duration = row['duration'] #first row of location.lat column here
  if math.isnan(provided_duration) or provided_duration <= 0 :
    time_dif = (row['drop_time'] - row['pickup_time']).seconds
    if(time_dif == 0):
        time_dif = np.nan
    durations.append(time_dif)
  else :  
    durations.append(provided_duration)

training_df.insert(4,"time_dif",durations)

In [8]:
new_column = []                    #empty column for distance
for index,row in training_df.iterrows():
  lat1 = row['pick_lat'] #first row of location.lat column here
  lon1 = row['pick_lon'] #first row of location.long column here
  lat2 = row['drop_lat'] #second row of location.lat column here
  lon2 = row['drop_lon'] #second row of location.long column here
  value = dist_from_coordinates(lat1, lon1, lat2, lon2)  #get the distance
  new_column.append(value)   #append the empty list with distance values

training_df.insert(4,"distance",new_column)

In [9]:
training_df['time_driven'] = training_df['duration']  - training_df['meter_waiting']

In [10]:
chargeperhours = []
for index,row in training_df.iterrows():
    if(row['meter_waiting'] == 0):
        chargeperhour = np.nan
    else:
        chargeperhour = (row['meter_waiting_fare'] / row['meter_waiting'] * 3600)
    chargeperhours.append(chargeperhour)

training_df.insert(4,'charge_per_hour',chargeperhours)


In [11]:
training_df['driving_fare'] = training_df['fare']  - training_df['meter_waiting_fare'] - training_df['additional_fare']

In [12]:
avgspeeds = []
for index,row in training_df.iterrows():
    if(row['time_driven'] == 0):
        avgspeed = np.nan
    else:
        avgspeed = (row['distance'] / row['time_driven'] * 3600)
    avgspeeds.append(avgspeed)

training_df.insert(4,"avg_speed",avgspeeds)



In [13]:
costsperkm = []
for index,row in training_df.iterrows():
    if row['distance'] == 0:
        costperkm = np.nan
            
    else:
        costperkm = (row['driving_fare'] / row['distance'])
    costsperkm.append(costperkm)

training_df.insert(4,"cost_per_km",costsperkm)

In [14]:
training_df = training_df.replace({'label': {'incorrect': 0, 'correct' : 1}})

In [15]:
training_df.columns

Index(['additional_fare', 'duration', 'meter_waiting', 'meter_waiting_fare',
       'cost_per_km', 'avg_speed', 'charge_per_hour', 'distance', 'time_dif',
       'meter_waiting_till_pickup', 'pickup_time', 'drop_time', 'pick_lat',
       'pick_lon', 'drop_lat', 'drop_lon', 'fare', 'label', 'timeOfDay',
       'isNormalCharge', 'time_driven', 'driving_fare'],
      dtype='object')

In [16]:
training_columns = ['meter_waiting','meter_waiting_fare','fare','additional_fare', 'distance','cost_per_km', 'avg_speed',  'time_dif','time_driven', 'charge_per_hour', 'driving_fare', 'isNormalCharge','pick_lat','pick_lon','drop_lat','drop_lon']

In [17]:
target_column = ['label']

In [18]:
training_df = training_df.drop(190167541)

In [19]:
x = training_df[training_columns].values
y = training_df[target_column].values

In [20]:
x, x_test, y, y_test = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)

In [21]:
training_df['label'].value_counts()

1    15494
0     1681
Name: label, dtype: int64

<h2>Choosing the best model</h2>

In [22]:
def focal_loss_lgb_eval_error(y_true, y_pred, alpha=.25, gamma=2.):
    a,g = alpha, gamma
    p = 1/(1+np.exp(-y_pred))
    loss = -( a*y_true + (1-a)*(1-y_true) ) * (( 1 - ( y_true*p + (1-y_true)*(1-p)) )**g) * ( y_true*np.log(p)+(1-y_true)*np.log(1-p) )
    return 'focal_loss', np.mean(loss), False

In [23]:
from sklearn.metrics import f1_score
def evaluate_macroF1_lgb(y_true, y_pred):  
    y_hat = np.where(y_pred < 0.5, 0, 1) 
    f1 = f1_score(y_true, y_hat, average='macro')
    return ('macroF1', f1, True) 

In [24]:
gkf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [26]:
model = lightgbm.LGBMClassifier(class_weight={0:1.25,1:1}, learning_rate=0.1)

In [27]:
for training_index, testing_index in gkf.split(X=x, y=y):
    x_train_fold, y_train_fold = x[training_index], y[training_index]
    x_test_fold, y_test_fold = x[testing_index], y[testing_index]
    model.fit(x_train_fold, y_train_fold, eval_set=(x_test_fold,y_test_fold),eval_metric = lambda y_true, y_pred: [evaluate_macroF1_lgb(y_true,y_pred)])

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[1]	valid_0's binary_logloss: 0.279743	valid_0's macroF1: 0.474268
[2]	valid_0's binary_logloss: 0.256452	valid_0's macroF1: 0.474268
[3]	valid_0's binary_logloss: 0.239624	valid_0's macroF1: 0.474268
[4]	valid_0's binary_logloss: 0.227107	valid_0's macroF1: 0.474268
[5]	valid_0's binary_logloss: 0.216238	valid_0's macroF1: 0.675978
[6]	valid_0's binary_logloss: 0.206864	valid_0's macroF1: 0.728548
[7]	valid_0's binary_logloss: 0.198954	valid_0's macroF1: 0.791522
[8]	valid_0's binary_logloss: 0.19217	valid_0's macroF1: 0.805833
[9]	valid_0's binary_logloss: 0.186306	valid_0's macroF1: 0.816965
[10]	valid_0's binary_logloss: 0.181381	valid_0's macroF1: 0.82517
[11]	valid_0's binary_logloss: 0.176879	valid_0's macroF1: 0.830061
[12]	valid_0's binary_logloss: 0.173307	valid_0's macroF1: 0.836483
[13]	valid_0's binary_logloss: 0.169471	valid_0's macroF1: 0.841637
[14]	valid_0's binary_logloss: 0.167151	valid_0's macroF1: 0.83806
[15]	valid_0's binary_logloss: 0.164392	valid_0's macroF1: 0

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[1]	valid_0's binary_logloss: 0.28237	valid_0's macroF1: 0.474268
[2]	valid_0's binary_logloss: 0.260641	valid_0's macroF1: 0.474268
[3]	valid_0's binary_logloss: 0.24408	valid_0's macroF1: 0.474268
[4]	valid_0's binary_logloss: 0.231841	valid_0's macroF1: 0.474268
[5]	valid_0's binary_logloss: 0.221169	valid_0's macroF1: 0.667272
[6]	valid_0's binary_logloss: 0.212632	valid_0's macroF1: 0.692993
[7]	valid_0's binary_logloss: 0.204097	valid_0's macroF1: 0.758875
[8]	valid_0's binary_logloss: 0.19791	valid_0's macroF1: 0.781274
[9]	valid_0's binary_logloss: 0.19169	valid_0's macroF1: 0.801454
[10]	valid_0's binary_logloss: 0.186831	valid_0's macroF1: 0.806675
[11]	valid_0's binary_logloss: 0.18249	valid_0's macroF1: 0.810105
[12]	valid_0's binary_logloss: 0.179029	valid_0's macroF1: 0.809151
[13]	valid_0's binary_logloss: 0.175555	valid_0's macroF1: 0.81083
[14]	valid_0's binary_logloss: 0.172779	valid_0's macroF1: 0.810723
[15]	valid_0's binary_logloss: 0.169922	valid_0's macroF1: 0.81

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[27]	valid_0's binary_logloss: 0.155174	valid_0's macroF1: 0.836289
[28]	valid_0's binary_logloss: 0.154383	valid_0's macroF1: 0.840318
[29]	valid_0's binary_logloss: 0.153964	valid_0's macroF1: 0.841177
[30]	valid_0's binary_logloss: 0.153741	valid_0's macroF1: 0.840933
[31]	valid_0's binary_logloss: 0.153124	valid_0's macroF1: 0.842397
[32]	valid_0's binary_logloss: 0.152769	valid_0's macroF1: 0.840933
[33]	valid_0's binary_logloss: 0.152829	valid_0's macroF1: 0.842397
[34]	valid_0's binary_logloss: 0.152592	valid_0's macroF1: 0.840318
[35]	valid_0's binary_logloss: 0.152481	valid_0's macroF1: 0.840079
[36]	valid_0's binary_logloss: 0.152555	valid_0's macroF1: 0.840933
[37]	valid_0's binary_logloss: 0.152622	valid_0's macroF1: 0.842397
[38]	valid_0's binary_logloss: 0.152059	valid_0's macroF1: 0.844715
[39]	valid_0's binary_logloss: 0.15126	valid_0's macroF1: 0.844116
[40]	valid_0's binary_logloss: 0.150972	valid_0's macroF1: 0.845577
[41]	valid_0's binary_logloss: 0.150528	valid_0's

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[1]	valid_0's binary_logloss: 0.280216	valid_0's macroF1: 0.474268
[2]	valid_0's binary_logloss: 0.258215	valid_0's macroF1: 0.474268
[3]	valid_0's binary_logloss: 0.24134	valid_0's macroF1: 0.474268
[4]	valid_0's binary_logloss: 0.227581	valid_0's macroF1: 0.48183
[5]	valid_0's binary_logloss: 0.21563	valid_0's macroF1: 0.661493
[6]	valid_0's binary_logloss: 0.20543	valid_0's macroF1: 0.744995
[7]	valid_0's binary_logloss: 0.196932	valid_0's macroF1: 0.783149
[8]	valid_0's binary_logloss: 0.189591	valid_0's macroF1: 0.807598
[9]	valid_0's binary_logloss: 0.183296	valid_0's macroF1: 0.815351
[10]	valid_0's binary_logloss: 0.178264	valid_0's macroF1: 0.81865
[11]	valid_0's binary_logloss: 0.17362	valid_0's macroF1: 0.823392
[12]	valid_0's binary_logloss: 0.169823	valid_0's macroF1: 0.824881
[13]	valid_0's binary_logloss: 0.166428	valid_0's macroF1: 0.832699
[14]	valid_0's binary_logloss: 0.162765	valid_0's macroF1: 0.832699
[15]	valid_0's binary_logloss: 0.159626	valid_0's macroF1: 0.83

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[1]	valid_0's binary_logloss: 0.282186	valid_0's macroF1: 0.474268
[2]	valid_0's binary_logloss: 0.26021	valid_0's macroF1: 0.474268
[3]	valid_0's binary_logloss: 0.244782	valid_0's macroF1: 0.474268
[4]	valid_0's binary_logloss: 0.232744	valid_0's macroF1: 0.474268
[5]	valid_0's binary_logloss: 0.222414	valid_0's macroF1: 0.507028
[6]	valid_0's binary_logloss: 0.212703	valid_0's macroF1: 0.741297
[7]	valid_0's binary_logloss: 0.204984	valid_0's macroF1: 0.77413
[8]	valid_0's binary_logloss: 0.197849	valid_0's macroF1: 0.79792
[9]	valid_0's binary_logloss: 0.191917	valid_0's macroF1: 0.797961
[10]	valid_0's binary_logloss: 0.186979	valid_0's macroF1: 0.813572
[11]	valid_0's binary_logloss: 0.181946	valid_0's macroF1: 0.818549
[12]	valid_0's binary_logloss: 0.177636	valid_0's macroF1: 0.823526
[13]	valid_0's binary_logloss: 0.173907	valid_0's macroF1: 0.830061
[14]	valid_0's binary_logloss: 0.171031	valid_0's macroF1: 0.831677
[15]	valid_0's binary_logloss: 0.168248	valid_0's macroF1: 0

In [28]:
predicted_y = model.predict(x_test)
train_pred_y = model.predict(x)

In [29]:
print(); print(metrics.classification_report(y,train_pred_y))
print(); print(metrics.confusion_matrix(y, train_pred_y))

print(); print(metrics.classification_report(y_test, predicted_y))
print(); print(metrics.confusion_matrix(y_test, predicted_y))


              precision    recall  f1-score   support

           0       0.96      0.84      0.89      1345
           1       0.98      1.00      0.99     12395

    accuracy                           0.98     13740
   macro avg       0.97      0.92      0.94     13740
weighted avg       0.98      0.98      0.98     13740


[[ 1125   220]
 [   50 12345]]

              precision    recall  f1-score   support

           0       0.87      0.68      0.76       336
           1       0.97      0.99      0.98      3099

    accuracy                           0.96      3435
   macro avg       0.92      0.84      0.87      3435
weighted avg       0.96      0.96      0.96      3435


[[ 229  107]
 [  35 3064]]


<h2>Testing</h2>

In [30]:

test_set = pd.read_csv(data_path+'test.csv', index_col="tripid")


In [31]:
test_set['pickup_time'] = pd.to_datetime(test_set['pickup_time'], format="%m/%d/%Y %H:%M")
test_set['drop_time'] = pd.to_datetime(test_set['drop_time'], format="%m/%d/%Y %H:%M")

In [32]:
test_set = test_set.assign(timeOfDay=pd.cut(test_set.pickup_time.dt.hour,[-1, 8, 20, 24],labels=['dawn','day', 'night']))

In [33]:
test_set.loc[test_set['timeOfDay'] == 'day', 'isNormalCharge'] = 1
test_set.loc[test_set['timeOfDay'] != 'day', 'isNormalCharge'] = 0

In [34]:
durations = []
for index,row in test_set.iterrows():
  provided_duration = row['duration'] #first row of location.lat column here
  if math.isnan(provided_duration) or provided_duration <= 0 :
    time_dif = (row['drop_time'] - row['pickup_time']).seconds
    if(time_dif == 0):
        time_dif = np.nan
    durations.append(time_dif)
  else :  
    durations.append(provided_duration)

test_set.insert(4,"time_dif",durations)

In [35]:
new_column = []                    #empty column for distance
for index,row in test_set.iterrows():
  lat1 = row['pick_lat'] #first row of location.lat column here
  lon1 = row['pick_lon'] #first row of location.long column here
  lat2 = row['drop_lat'] #second row of location.lat column here
  lon2 = row['drop_lon'] #second row of location.long column here
  value = dist_from_coordinates(lat1, lon1, lat2, lon2)  #get the distance
  new_column.append(value)   #append the empty list with distance values

test_set.insert(4,"distance",new_column)

In [36]:
test_set['time_driven'] = test_set['duration']  - test_set['meter_waiting']

In [37]:
chargeperhours = []
for index,row in test_set.iterrows():
    if(row['meter_waiting'] == 0):
        chargeperhour = np.nan
    else:
        chargeperhour = (row['meter_waiting_fare'] / row['meter_waiting'] * 3600)
    chargeperhours.append(chargeperhour)

test_set.insert(4,'charge_per_hour',chargeperhours)


In [38]:
test_set['driving_fare'] = test_set['fare']  - test_set['meter_waiting_fare'] - test_set['additional_fare']

In [39]:
avgspeeds = []
for index,row in test_set.iterrows():
    if(row['time_driven'] == 0):
        avgspeed = np.nan    
    else:
        avgspeed = (row['distance'] / row['time_driven'] * 3600)
    avgspeeds.append(avgspeed)

test_set.insert(4,"avg_speed",avgspeeds)



In [40]:
costsperkm = []
for index,row in test_set.iterrows():
    if row['distance'] == 0:
        costperkm = np.nan
            
    else:
        costperkm = (row['driving_fare'] / row['distance'])
    costsperkm.append(costperkm)

test_set.insert(4,"cost_per_km",costsperkm)

In [41]:
test_features = test_set[training_columns]

In [42]:
test_features.isna().sum()

meter_waiting           0
meter_waiting_fare      0
fare                    0
additional_fare         0
distance                0
cost_per_km            25
avg_speed               9
time_dif                0
time_driven             0
charge_per_hour       298
driving_fare            0
isNormalCharge          0
pick_lat                0
pick_lon                0
drop_lat                0
drop_lon                0
dtype: int64

In [43]:
predicted_labels = model.predict(test_features)

In [44]:
predicted_labels_df = pd.DataFrame(predicted_labels )

In [45]:
sub_path =os.path.abspath(os.path.join(data_path+'/sample_submission.csv'))
submission_set = pd.read_csv(sub_path, index_col="tripid")

In [46]:
submission_set['prediction']= predicted_labels_df.values[:,0]

In [47]:
%%javascript
var kernel = IPython.notebook.kernel;
var thename = window.document.getElementById("notebook_name").innerHTML;
var command = "theNotebook = " + "'"+thename+"'";
kernel.execute(command);

<IPython.core.display.Javascript object>

In [49]:
filename = '../../submissions/'+theNotebook+'/'+theNotebook+'_{%i}.csv'
dirname = '../../submissions/'+theNotebook
fileversion = 1

if not os.path.exists(dirname):
    os.makedirs(dirname)
while glob.glob(filename.replace('{%i}',str(fileversion))) :
    fileversion+=1
submission_set.to_csv(filename.replace('{%i}',str(fileversion)), index=True)
print("Completed!")

Completed!


In [48]:
submission_set['prediction'].value_counts()

1    8138
0     438
Name: prediction, dtype: int64